# Part 1


As you look out the window and notice a heavily-forested continent slowly appear over the horizon, you are interrupted by the child sitting next to you. They're curious if you could help them with their math homework.

Unfortunately, it seems like this "math" follows different rules than you remember.

The homework (your puzzle input) consists of a series of expressions that consist of addition (`+`), multiplication (`*`), and parentheses (`(...)`). Just like normal math, parentheses indicate that the expression inside must be evaluated before it can be used by the surrounding expression. Addition still finds the sum of the numbers on both sides of the operator, and multiplication still finds the product.

However, the rules of operator precedence have changed. Rather than evaluating multiplication before addition, the operators have the same precedence, and are evaluated left-to-right regardless of the order in which they appear.

For example, the steps to evaluate the expression `1 + 2 * 3 + 4 * 5 + 6` are as follows:
```
1 + 2 * 3 + 4 * 5 + 6
  3   * 3 + 4 * 5 + 6
      9   + 4 * 5 + 6
         13   * 5 + 6
             65   + 6
                 71
```
Parentheses can override this order; for example, here is what happens if parentheses are added to form `1 + (2 * 3) + (4 * (5 + 6))`:
```
1 + (2 * 3) + (4 * (5 + 6))
1 +    6    + (4 * (5 + 6))
     7      + (4 * (5 + 6))
     7      + (4 *   11   )
     7      +     44
            51
```

Here are a few more examples:
```
2 * 3 + (4 * 5) becomes 26.
5 + (8 * 3 + 9 + 3 * 4 * 3) becomes 437.
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) becomes 12240.
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 becomes 13632.
```

Before you can help with the homework, you need to understand it yourself. Evaluate the expression on each line of the homework; what is the sum of the resulting values?

In [1]:
from operator import add, mul


_OPS = {'+': add, '*': mul}


class Op:
    def __init__(self, operator, left_hand, right_hand):
        if operator not in _OPS:
            raise Exception('Operator not supported: %s.' % operator)
        self.op = operator
        self.left_h = left_hand
        self.right_h = right_hand
    
    def __repr__(self):
        return '%s(%s, %s)' % (self.op, self.left_h, self.right_h)
    
    def compute(self):
        op = _OPS[self.op]
        left_h = self.left_h if isinstance(self.left_h, int) else self.left_h.compute()
        right_h = self.right_h if isinstance(self.right_h, int) else self.right_h.compute()
        return op(left_h, right_h)

    
# Test
op = Op('+', 2, Op('*', 3,  4))
assert op.compute() == 2 + (3 * 4)


def get_input():
    with open('inputs/18') as f:
        return f.read().splitlines()
    
    
def tokenize(line):
    it = iter(line)
    return _tokenize(it)


def _tokenize(iterator):
    result = []
    tok = ''
    while True:
        try:
            ch = next(iterator)
        except StopIteration:
            if tok != '':
                result.append(convert_if_int(tok))
            return result
        if ch == '(':
            tok = _tokenize(iterator)
            result.append(convert_if_int(tok))
            tok = ''
        elif ch == ')':
            if tok != '':
                result.append(convert_if_int(tok))
            return result
        elif ch == ' ':
            if tok != '':
                result.append(convert_if_int(tok))
            tok = ''
        else:
            tok += ch
        

def convert_if_int(s):
    try:
        return int(s)
    except:
        return s
    

# Test
test_line = '1 + (2 * 3) + (4 * (5 + 6))'
assert tokenize(test_line) == [1, '+', [2, '*', 3], '+', [4, '*', [5, '+', 6]]]


def parse(token_lst):
    elems = {'left_hand': '', 'operator': '', 'right_hand': ''}
    current = 'left_hand'
    for tok in token_lst:
        el = parse(tok) if type(tok) is list else tok
        elems[current] = el
        if current == 'left_hand':
            current = 'operator'
        elif current == 'operator':
            current = 'right_hand'
        elif current == 'right_hand':
            op = Op(**elems)
            elems['left_hand'] = op
            current = 'operator'
    return elems['left_hand']


# Test
op = parse([1, '+', [2, '*', 3], '+', [4, '*', [5, '+', 6]]])
assert op.compute() == 51


def evaluate_line(line):
    token_lst = tokenize(line)
    op = parse(token_lst)
    return op.compute()


# Test
assert evaluate_line('1 + (2 * 3) + (4 * (5 + 6))') == 51


def result1():
    return sum(evaluate_line(line) for line in get_input())


result1()

510009915468

# Part 2

You manage to answer the child's questions and they finish part 1 of their homework, but get stuck when they reach the next section: advanced math.

Now, addition and multiplication have different precedence levels, but they're not the ones you're familiar with. Instead, addition is evaluated before multiplication.

For example, the steps to evaluate the expression `1 + 2 * 3 + 4 * 5 + 6` are now as follows:
```
1 + 2 * 3 + 4 * 5 + 6
  3   * 3 + 4 * 5 + 6
  3   *   7   * 5 + 6
  3   *   7   *  11
     21       *  11
         231
```

Here are the other examples from above:
```
1 + (2 * 3) + (4 * (5 + 6)) still becomes 51.
2 * 3 + (4 * 5) becomes 46.
5 + (8 * 3 + 9 + 3 * 4 * 3) becomes 1445.
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) becomes 669060.
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 becomes 23340.
```
What do you get if you add up the results of evaluating the homework problems using these new rules?

In [2]:
_OP_PRECEDENCE = ['+', '*']


def add_parantheses(token_lst):
    current = token_lst
    # No need to add parentheses for the op with lowest precedence.
    # This actually makes this function and _OP_PRECEDENCE unnecessary since we only have
    # 2 operations and thus could just call `_add_parenthese_to_op(token_lst, '+')`, but in the
    # spirit of making it more real-like, I've left it like this.
    for op_sign in _OP_PRECEDENCE[:-1]:
        current = _add_parantheses_to_op(current, op_sign)
    return current


def _add_parantheses_to_op(token_lst, op_sign):
    result = []
    
    def recursive_next(it):
        n = next(it)
        if type(n) is list:
            return _add_parantheses_to_op(n, op_sign)
        return n
             
    it = iter(token_lst)
    prev = recursive_next(it)
    while True:
        try:
            curr = recursive_next(it)
        except StopIteration:
            result.append(prev)
            return result

        if curr == op_sign:
            right = recursive_next(it)
            prev = [prev, curr, right]
        else:
            result.append(prev)
            prev = curr
    return result


def parse2(token_lst):
    token_lst = add_parantheses(token_lst)
    return parse(token_lst)


def evaluate_line2(line):
    token_lst = tokenize(line)
    op = parse2(token_lst)
    return op.compute()


# Test
assert evaluate_line2('1 + 2 * 3 + 4 * 5 + 6') == 231
assert evaluate_line2('1 + (2 * 3) + (4 * (5 + 6))') == 51
assert evaluate_line2('2 * 3 + (4 * 5)') == 46
assert evaluate_line2('5 + (8 * 3 + 9 + 3 * 4 * 3)') == 1445
assert evaluate_line2('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))') == 669060
assert evaluate_line2('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2') == 23340


def result2():
    return sum(evaluate_line2(line) for line in get_input())


result2()

321176691637769